# Bài tập 1

In [ ]:
!gdown --id 1pESuZ_SjtR-kcxokNKuBwaSufVtsQCql

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1pESuZ_SjtR-kcxokNKuBwaSufVtsQCql
To: /content/vietnam.mp4
100% 5.47M/5.47M [00:00<00:00, 37.9MB/s]


In [ ]:
!pip install ultralytics

## simple version

In [ ]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

In [ ]:
model = YOLO("yolo11l.pt")

video_path = "./vietnam.mp4"
cap = cv2.VideoCapture(video_path)

100%|██████████| 49.0M/49.0M [00:00<00:00, 276MB/s]


In [ ]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

video_name = video_path.split("/")[-1]
output_path = f"./{video_name.split('.')[0]}_tracked.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path,fourcc,fps,(width,height))

In [ ]:
track_history = defaultdict(lambda:[])

while cap.isOpened():

  success,frame =cap.read()

  if success:
    results = model.track(frame,persist=True,show=False)

    boxes = results[0].boxes.xywh.cpu()
    try:
      track_ids = results[0].boxes.id
      if track_ids is not None:
        track_ids = track_ids.int().cpu().tolist()
      else:
        track_ids = []
    except AttributeError:

      track_ids = []
    annotated_frame = results[0].plot()

    if track_ids:
      for box,track_id in zip(boxes,track_ids):
        x,y,w,h = box
        track = track_history[track_id]
        track.append((float(x),float(y)))

        if len(track)> 120:
          track.pop(0)
        points = np.hstack(track).astype(np.int32).reshape((-1,1,2))
        cv2.polylines(
            annotated_frame,
            [points],
            isClosed=False,
            color = (230,230,230),
            thickness=4,
        )
      out.write(annotated_frame)
  else:
      break
cap.release()
out.release()
print(f"Video has been saved to {output_path}")


Video has been saved to ./vietnam_tracked.mp4


## optimized version

In [ ]:
!pip install loguru

In [ ]:
import argparse
from collections import defaultdict
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from loguru import logger

In [ ]:
def load_config():
  return {
      "model_path":"yolo11x.pt",
      "track_history_length":120,
      "batch_size":64,
      "line_thickness":4,
      "track_color":(230,230,230)
  }
def initialize_video(video_path):
  cap = cv2.VideoCapture(video_path)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))

  video_name = video_path.split("/")[-1]
  output_path = f"./{video_name.split('.')[0]}_tracked.mp4"
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")
  out = cv2.VideoWriter(output_path,fourcc,fps,(width,height))
  return cap,out,output_path

In [ ]:
from threading import current_thread
def update_track_history(
    track_history,
    last_seen,
    track_ids,
    frame_count,
    batch_size,
    frame_idx,
    history_length
):
  current_tracks = set(track_ids)
  for track_id in list(track_history.keys()):
    if track_id in current_tracks:
      last_seen[track_id] = frame_count - (batch_size-frame_idx-1)
    elif frame_count-last_seen[track_id] > history_length:
      del track_history[track_id]
      del last_seen[track_id]


In [ ]:
def draw_tracks(frame,boxes,track_ids,track_history,config):
  if not track_ids:
    return frame
  for box,track_id in zip(boxes,track_ids):
    x,y,w,h = box
    track = track_history[track_id]
    track.append((float(x),float(y)))
    if len(track) > config['track_history_length']:
      track.pop(0)
    points = np.hstack(track).astype(np.int32).reshape((-1,1,2))
    cv2.polylines(
        frame,
        [points],
        isClosed=False,
        color=config['track_color'],
        thickness=config['line_thickness']
    )
  return frame

In [ ]:
def process_batch(model,batch_frames,track_history,last_seen,frame_count,config):
  results = model.track(batch_frames,persist=True,tracker="botsort.yaml",show=False,verbose=False,iou=0.5)
  processed_frames=[]
  for frame_idx,result in enumerate(results):
    boxes = result.boxes.xywh.cpu()
    track_ids = (result.boxes.id.int().tolist() if result.boxes.id is not None else [])

    update_track_history(track_history,last_seen,track_ids,frame_count,len(batch_frames),frame_idx,config['track_history_length'])

    annotated_frame = result.plot(font_size=4,line_width=2)
    annotated_frame = draw_tracks(
        annotated_frame,boxes,track_ids,track_history,config
    )
    processed_frames.append(annotated_frame)
  return processed_frames

In [ ]:
def main(video_path):

  CONFIG = load_config()
  model = YOLO(CONFIG.get("model_path","yolo11x.pt"))
  cap,out,output_path = initialize_video(video_path)
  track_history = defaultdict(lambda:[])

  last_seen = defaultdict(int)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  with tqdm(total=total_frames,desc="Processing frames",colour="green") as pbar:
    frame_count = 0
    batch_frames = []

    while cap.isOpened():
      success,frame=cap.read()
      if not success:
        break
      batch_frames.append(frame)
      frame_count += 1

      if len(batch_frames) == CONFIG['batch_size'] or frame_count == total_frames:
        try:
          processed_frames = process_batch(model,batch_frames,track_history,last_seen,frame_count,CONFIG)
          for frame in processed_frames:
            out.write(frame)
            pbar.update(1)
          batch_frames = []
        except Exception as e:
          logger.error(f"Error when handling frames {frame_count-len(batch_frames)+1} to {frame_count}: {str(e)}")
          batch_frames = []
          continue
    try:
      cap.release()
      out.release()
      cv2.destroyAllWindows()
      logger.info(f"{output_path}")
    except Exception as e:
      logger.error(f"{str(e)}")
main("./vietnam.mp4")
# if __name__ == "__main__":
#   parser = argparse.ArgumentParser()
#   parser.add_argument("--video-path",type=str,default="samples/vietnam-2.mp4")
#   args = parser.parse_args()
#   main(args.video_path)

100%|██████████| 109M/109M [00:00<00:00, 213MB/s]
Processing frames: 100%|██████████| 780/780 [01:40<00:00,  7.76it/s]


In [ ]:
!python main.py --video-path "samples/vietnam-2.mp4"

python3: can't open file '/content/main.py': [Errno 2] No such file or directory


# Bài tập 2

In [ ]:
!gdown --id 1a4vuMZJOC8MGpy_tcRDsmt8-yPtAIW9q

In [ ]:
from ultralytics import solutions
import cv2

In [ ]:
cap = cv2.VideoCapture("./highway.mp4")
assert cap.isOpened(), "Error reading video file"
w,h,fps = (
    int(cap.get(x))
    for x in (cv2.CAP_PROP_FRAME_WIDTH,cv2.CAP_PROP_FRAME_HEIGHT,cv2.CAP_PROP_FPS)
)


In [ ]:
region_points = [
    (430,700),
    (1600,700),
    (1600,1080),
    (430,1080)
]

In [ ]:
video_writer = cv2.VideoWriter(
    "./highway_counted.mp4", cv2.VideoWriter_fourcc(*"mp4v"),fps,(w,h)
)

In [ ]:
counter = solutions.ObjectCounter(
    show = False,
    region = region_points,
    model="yolo11x.pt"
)

In [ ]:
while cap.isOpened():
  success,in0 = cap.read()
  if not success:
    print("Video frame is empty or video processing has been successfully completed")
    break
  in0 = counter.count(in0)
  video_writer.write(in0)
cap.release()
video_writer.release()
cv2.destroyAllWindows()

# Bài tập 3

In [ ]:
!gdown --id 11qqiZsW6nqB_VHG42mwCdgA_ASrSu3nr

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11qqiZsW6nqB_VHG42mwCdgA_ASrSu3nr
To: /content/bus.jpg
100% 137k/137k [00:00<00:00, 107MB/s]


In [ ]:
!pip install ultralytics

In [ ]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLOWorld
from ultralytics.engine.results import Boxes
import sys
import uuid
from pathlib import Path
import cv2
from loguru import logger



In [ ]:
def save_detection_results(results):

  saved_paths = []

  for i,result in enumerate(results):

    if len(result.boxes) > 0:
      annotated_image = result.plot()

      output_path = f"./img_{uuid.uuid4()}.jpg"

      cv2.imwrite(output_path,annotated_image)

      saved_path = Path(output_path).resolve()
      print(f"Image saved to {saved_path}")
      saved_paths.append(str(saved_path))
  return saved_paths

In [ ]:
model = YOLOWorld("yolov8s-world.pt")
model.set_classes(['bus'])
results: Boxes = model.predict("bus.jpg")


image 1/1 /content/bus.jpg: 640x480 1 bus, 21.3ms
Speed: 8.8ms preprocess, 21.3ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
save_detection_results(results)

Image saved to /content/img_82f58d30-974b-4b94-84b5-322803e78eb5.jpg


['/content/img_82f58d30-974b-4b94-84b5-322803e78eb5.jpg']